## Python version of get_warped_xy_mmask.m

### Import modules

In [252]:
# make sure to conda install imread, cv2

import os
import sys
import numpy as np
import cv2
import numpy.matlib
import dlib

from scipy.io import loadmat
from scipy.ndimage import imread, affine_transform


### Add data folder to path and import .mat file

In [87]:
tracker_path = r'data/images_tracker/00047'
reftracker = loadmat(tracker_path)['tracker']


### Normalize x and y channels

In [253]:

def get_facial_points(image, num_points):
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    detector = dlib.get_frontal_face_detector()
    dets = detector(image, 1)
    points = []
    for k, d in enumerate(dets):
        # Get the landmarks/parts for the face in box d.
        shape = predictor(image, d)
        for i in range(num_points):
            pt = shape.part(i)
            points.append([int(pt.x), int(pt.y)])
    return np.array(points)

def im2double(im):
    info = np.iinfo(im.dtype) # Get the data type of the input image
    return im.astype(np.float) / info.max # Divide all values by the largest possible value in the datatype

def normalize_image(imat):
    rgb = np.zeros((imat.shape[0], imat.shape[1], 3), dtype=np.int)
    rgb[:, :, 0] = np.round((imat[:, :, 2] + 122.675)/255)
    rgb[:, :, 1] = np.round((imat[:, :, 1] + 116.669)/255)
    rgb[:, :, 2] = np.round((imat[:, :, 0] + 104.008)/255)

refpos = np.floor(reftracker.mean(axis=0))

xxc, yyc = np.meshgrid(np.linspace(1,1800, 1800), np.linspace(1,2000, 2000))
xxc = (xxc-600-refpos[0])/600
yyc = (yyc-600-refpos[1])/800

In [89]:
mmask_png =  r'meanmask.png'
maskc = im2double(imread(mmask_png))
print(np.mean(maskc[0]))
maskc = np.pad(maskc, [600,600], 'constant')

0.3671111111111111


### Conditional on destracker

In [90]:
destracker_path = r'data/images_tracker/00020'
destracker = loadmat(destracker_path)['tracker']
image = loadmat(r'data/portraitFCN+_data/00020')['img']
img_true = image[:, :, 0:3]
warpedxx_true = image[:, :, 3]
warpedyy_true = image[:, :, 4]
warpedmask_true = image[:, :, 5]

if np.shape(destracker)[0] == 49:
    print('ree')


ree


In [219]:
import nudged

In [238]:
# tform = cv2.getAffineTransform(pts1, pts2)
h, status = cv2.findHomography(a, b)

In [239]:
trans = nudged.estimate(a, b);
h = np.array(trans.get_matrix())[0:2]

In [240]:
# affine_transform(a, b)
warpedxx = np.array(cv2.warpAffine(xxc, h, xxc.shape))
warpedyy = np.array(cv2.warpAffine(yyc, h, yyc.shape))
warpedmask = np.array(cv2.warpAffine(maskc, h, maskc.shape))


In [241]:
warpedxx = warpedxx[600:1400, 600:1200]
warpedyy = warpedyy[600:1400, 600:1200]
warpedmask = warpedmask[600:1400, 600:1200]
        

In [250]:
e = warpedxx-warpedxx_true

In [251]:
np.mean(e.flatten())

0.04152811697909831

In [255]:
rgb = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
rgb[:, :, 0] = np.round(img[:, :, 2] * 255 + 122.675)
rgb[:, :, 1] = np.round(img[:, :, 1] * 255 + 116.669)
rgb[:, :, 2] = np.round(img[:, :, 0] * 255 + 104.008)
get_facial_points(rgb, 49)


array([[254, 389],
       [257, 424],
       [264, 456],
       [273, 487],
       [284, 515],
       [302, 541],
       [323, 563],
       [346, 581],
       [376, 587],
       [405, 581],
       [432, 563],
       [453, 540],
       [469, 514],
       [480, 485],
       [486, 454],
       [491, 421],
       [491, 388],
       [261, 364],
       [272, 351],
       [291, 345],
       [310, 347],
       [327, 357],
       [392, 356],
       [412, 343],
       [436, 341],
       [457, 349],
       [473, 363],
       [359, 388],
       [358, 415],
       [357, 441],
       [356, 467],
       [339, 480],
       [349, 483],
       [361, 486],
       [374, 481],
       [387, 478],
       [284, 395],
       [297, 384],
       [316, 385],
       [331, 400],
       [315, 405],
       [296, 404],
       [397, 398],
       [413, 382],
       [432, 380],
       [448, 391],
       [435, 399],
       [415, 401],
       [323, 510]])

In [256]:
destracker

array([[264.7586 , 358.70364],
       [275.513  , 347.35376],
       [291.60327, 344.6247 ],
       [308.12665, 348.05914],
       [323.85873, 351.5761 ],
       [393.51923, 355.1911 ],
       [408.41528, 347.4593 ],
       [425.62332, 343.90247],
       [441.80576, 346.36078],
       [457.1753 , 352.8955 ],
       [361.72974, 393.6062 ],
       [363.8787 , 412.2971 ],
       [366.3925 , 430.4419 ],
       [369.73175, 448.58878],
       [351.90198, 473.2341 ],
       [359.53162, 477.0737 ],
       [368.77457, 478.47934],
       [380.22018, 476.3147 ],
       [389.9772 , 472.9381 ],
       [285.9214 , 395.84012],
       [301.16644, 386.24518],
       [317.21146, 388.55524],
       [330.28784, 400.73123],
       [315.32806, 405.4568 ],
       [299.52612, 404.94424],
       [395.69937, 398.53592],
       [409.96362, 385.0723 ],
       [427.85672, 382.3143 ],
       [441.0335 , 393.04187],
       [427.7671 , 400.72296],
       [411.5628 , 401.684  ],
       [335.06357, 512.85175],
       [